In [244]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import functools

data_path = 'Data/'

### Loading in test data
load data
organise info
clean up columns

In [282]:
choices = pd.read_csv(data_path + 'responses_so_far.csv')

In [283]:
agenda = [re.findall(r'\[([^\]]+)', col) for col in choices.columns[2:]]

In [284]:
regex = re.compile(".*?\((.*?)\)")
re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip())

'i'

In [285]:
new_cols = ['timestamp','name'] + [re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_') for co in agenda]

In [286]:
choices.columns = new_cols

In [287]:
schedule = [(co[0].replace(' ', '_'), re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_')) for co in agenda]

In [288]:
schedule_dict = {}
for pair in schedule:
    if pair[0] not in schedule_dict.keys():
        schedule_dict[pair[0]] = []
    schedule_dict[pair[0]].append(pair[1])

In [289]:
company_avai_dict = {re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_'):int(co[1].split('[')[1]) for co in agenda}

### Random allocation

Create dictionary that matches time slots to choices

In [290]:
choices = choices.drop(['timestamp'], axis = 1)

Trying out allocation

In [291]:
# dropping NA
choices = choices[~choices.name.isnull()].reset_index(drop = True)

In [292]:
def prize_draw(co, level, pool):
    limit = company_avai_dict[co]
    total_picks = sum(choices[co] == level)
    if total_picks > limit:
        return(list(np.random.choice(pool, limit, replace=False)))
    else:
        return(pool)

In [293]:
prize_draw_dict = {}
for slot, companies in schedule_dict.items():
    pool = list(choices.name)
    for co in schedule_dict['WEDNESDAY_LUNCH']:
        prize_draw_dict[co] = []
        for level in sorted(list(set(choices[co]))):
            winners = prize_draw(co, level, pool)
            company_avai_dict[co] = company_avai_dict[co] - len(winners)
            prize_draw_dict[co].extend(winners)
            pool = [company for company in pool if company not in winners]
            if company_avai_dict[co] == 0 or len(pool) == 0:
                print('all spots allocated for ', co)
                break

all spots allocated for  Google_Deepmind
all spots allocated for  Blockchain
all spots allocated for  Pierre_Richemond
all spots allocated for  Stéphane_Leroy
all spots allocated for  King.com
all spots allocated for  BNP_Paribas_Data_Analytics
all spots allocated for  Circle
all spots allocated for  Google_Deepmind
all spots allocated for  Pierre_Richemond
all spots allocated for  Stéphane_Leroy
all spots allocated for  King.com
all spots allocated for  BNP_Paribas_Data_Analytics
all spots allocated for  Circle
all spots allocated for  Google_Deepmind
all spots allocated for  Stéphane_Leroy
all spots allocated for  King.com
all spots allocated for  BNP_Paribas_Data_Analytics
all spots allocated for  Circle
all spots allocated for  Google_Deepmind
all spots allocated for  King.com
all spots allocated for  BNP_Paribas_Data_Analytics
all spots allocated for  Circle
all spots allocated for  Google_Deepmind
all spots allocated for  BNP_Paribas_Data_Analytics
all spots allocated for  Circle